In [ ]:
import json
import requests
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Access the API key
api_key = os.getenv("API_KEY")

print(api_key)  


# Get the Lagos, Nigeria weather data from the API
api_k = api_key
city = "Lagos,NG"
url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&cnt=240&appid={api_k}"

response = requests.get(url)
data = response.json()
json_str = json.dumps(data, indent=4)
print(json_str)


In [ ]:
#data = response.json()
#json_str = json.dumps(data,indent=4)
#print(json_str)

In [ ]:
# Create a function to convert temperature from Kelvin to Celsius
def kelvin_to_celsius(temp_k):
    return temp_k - 273.15

if response.status_code == 200:
    data = response.json()
    weather_data = []
    for hour in data['list']:
        date_time = datetime.utcfromtimestamp(hour['dt']).strftime('%Y-%m-%d %H:%M:%S')
        temp = kelvin_to_celsius(hour['main']['temp'])
        feels_like = kelvin_to_celsius(hour['main']['feels_like'])
        pressure = hour['main']['pressure']
        humidity = hour['main']['humidity']
        weather_main = hour['weather'][0]['main']
        weather_description = hour['weather'][0]['description']
        wind_speed = hour['wind']['speed']
        wind_direction = hour['wind']['deg']
        cloudiness = hour['clouds']['all']
        rain_volume = hour.get('rain', {}).get('3h', 0)
        snow_volume = hour.get('snow', {}).get('3h', 0)

        weather_data.append({
            "DateTime": date_time,
            "Temperature": temp,
            "Feels Like_temp": feels_like,
            "Pressure(hPa)": pressure,
            "Humidity_percent": humidity,
            "Weather": weather_main,
            "Weather Description": weather_description,
            "Wind Speed": wind_speed,
            "Wind Direction": wind_direction,
            "Cloudiness": cloudiness,
            "Rain Volume(mm)": rain_volume,
            "Snow Volume(mm)": snow_volume
        })

    df = pd.DataFrame(weather_data)
else:
            print(f"Failed to get data: {response.status_code}")
df.head()


In [ ]:
print(json_str)

In [ ]:
df.info()

In [ ]:
# Convert Datetime from Object to Datetime
df['DateTime'] = pd.to_datetime(df['DateTime'])

# Extract day and hour from Datetime
df['Day'] = df['DateTime'].dt.date
df['Hour'] = df['DateTime'].dt.hour

# Pivot the DataFrame
df_pivot = df.pivot(index='Day', columns='Hour', values='Temperature')

# Calculate average temperature
average_value = df['Temperature'].mean()

# Plot the data
plt.figure(figsize=(10, 2))
plt.plot(df['DateTime'], df['Temperature'], color='b', linestyle='-', linewidth=2, marker='o', markersize=4)
plt.axhline(y=average_value, color='r', linestyle='--', label=f'Average = {average_value:.2f}')  # Add average line
plt.title('Line Chart with Average Line')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend(loc='upper left')  # Add a Legend
plt.grid(True)
plt.show()




In [ ]:
plt.figure(figsize=(12, 3))
sns.heatmap(df_pivot, cmap='YlGnBu', annot=True, fmt=".1f", linewidths=.5)
plt.title('Day/Hour Heatmap')
plt.xlabel('Hour of the Day')
plt.ylabel('Day')
plt.show()


In [ ]:
# Import Libraries
import pandas as pd  # For data transformation, manipulation, analysis, and cleaning.
import psycopg2  # For connecting to PostgreSQL databases and executing queries.
from sqlalchemy import create_engine  # To efficiently manage and reuse database connections


# Database Credentials
db_username = 'postgres'
db_password = 'lana'
db_host = 'localhost' 
db_port = 5432
db_name = 'postgres'

# Establish a connection using SQLAlchemy
engine = create_engine(f'postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

# Create a table name
table_name = 'weather'

# Load the data into the table
df.to_sql(table_name, engine, if_exists='replace', index=False)

# Close the connection
engine.dispose()

